In [12]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2", base_url="host.docker.internal")

In [13]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

from plum_chatbot.configs.settings import Settings

settings = Settings()

client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="FAQ",
    embedding=embeddings,
)

/tmp/ipykernel_40317/340393869.py:8: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/plum_chatbot/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3.2", model_provider="ollama", base_url="host.docker.internal")

In [24]:
# vector_store.similarity_search("dati del profilo")
await vector_store.asearch("dati del profilo", search_type="mmr", k=2)

[Document(metadata={'_id': 1, '_collection_name': 'FAQ'}, page_content='2. COME MODIFICA I DATI DEL PROFILO\nDalla pagina iniziale premere il pulsante Profilo\nLa procedura visualizza quindi la finestra nella quale l’amministratore può inserire\ne/o modificare i propri dati\n E’ importante terminare l’operazione di inserimento/modifica premendo il pulsante\nsalva.\n'),
 Document(metadata={'_id': 5, '_collection_name': 'FAQ'}, page_content='1. COME CONFIGUARE LE EMAIL E LA PEC\nAll’interno del portale è possibile configurare tutti gli indirizzi email e Pec dello\nStudio  e  decidere,  a  seconda  delle  esigenze,  da  quale  indirizzo  inviare  la\ncorrispondenza.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Email \nPremere ora il simbolo \n  per inserire i parametri dell’email \nCompilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\noppure amministrazione, ecc.)\n- Tipo  (inserire per le email ‘OR

In [25]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict

# Define state for application
class State(TypedDict):
    question: str
    context: list[Document]
    answer: str


# Define application steps
async def retrieve(state: State):
    retrieved_docs = await vector_store.amax_marginal_relevance_search(state["question"], k=2)
    return {"context": retrieved_docs}


async def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = await prompt.ainvoke({"question": state["question"], "context": docs_content})
    response = await llm.ainvoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [28]:
response = await graph.ainvoke({"question": "come modificare i dati del profilo?"})
response

{'question': 'come modificare i dati del profilo?',
 'context': [Document(metadata={'_id': 1, '_collection_name': 'FAQ'}, page_content='2. COME MODIFICA I DATI DEL PROFILO\nDalla pagina iniziale premere il pulsante Profilo\nLa procedura visualizza quindi la finestra nella quale l’amministratore può inserire\ne/o modificare i propri dati\n E’ importante terminare l’operazione di inserimento/modifica premendo il pulsante\nsalva.\n'),
  Document(metadata={'_id': 5, '_collection_name': 'FAQ'}, page_content='1. COME CONFIGUARE LE EMAIL E LA PEC\nAll’interno del portale è possibile configurare tutti gli indirizzi email e Pec dello\nStudio  e  decidere,  a  seconda  delle  esigenze,  da  quale  indirizzo  inviare  la\ncorrispondenza.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Email \nPremere ora il simbolo \n  per inserire i parametri dell’email \nCompilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\n

In [44]:
import pandas as pd

pd.DataFrame(response)

,question,context,answer
0,come configurare la pec?,page_content='7. COME IMPOSTARE I DATI PER LA ...,Non ho informazioni sulla configurazione della...
1,come configurare la pec?,page_content='3. COME INSERIRE LE COORDINATE D...,Non ho informazioni sulla configurazione della...
2,come configurare la pec?,page_content='2. COME MODIFICA I DATI DEL PROF...,Non ho informazioni sulla configurazione della...
3,come configurare la pec?,page_content='1. COME CONFIGUARE LE EMAIL E LA...,Non ho informazioni sulla configurazione della...


In [42]:
print(response['answer'])

Non ho informazioni sulla configurazione della PEC (Poste Elettroniche Certified). Non conosco i dettagli esatti di come configrarla. Cerca i manuali o il supporto tecnico di PLUM.


In [33]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    vector_store.as_retriever(),
    "retrieve_faq",
    "Search and return information about frequently asked questions.",
)

In [34]:
retriever_tool.invoke("dati del profilo")

'2. COME MODIFICA I DATI DEL PROFILO\nDalla pagina iniziale premere il pulsante Profilo\nLa procedura visualizza quindi la finestra nella quale l’amministratore può inserire\ne/o modificare i propri dati\n E’ importante terminare l’operazione di inserimento/modifica premendo il pulsante\nsalva.\n\n\n1. COME CONFIGUARE LE EMAIL E LA PEC\nAll’interno del portale è possibile configurare tutti gli indirizzi email e Pec dello\nStudio  e  decidere,  a  seconda  delle  esigenze,  da  quale  indirizzo  inviare  la\ncorrispondenza.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Email \nPremere ora il simbolo \n  per inserire i parametri dell’email \nCompilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\noppure amministrazione, ecc.)\n- Tipo  (inserire per le email ‘ORDINARIA’ e per la pec ‘PEC’)\n- Indirizzo (riportare  l’indirizzo  dell’email  o  della  pec,  es:\nammistrazionerossi@gmail.com oppure info@st